In [1]:
#pip install selenium
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.chrome.service import Service

In [2]:
#pip install --upgrade selenium

In [3]:
chrome_options = webdriver.ChromeOptions()
# Add any additional options you need

service = Service(executable_path="C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")

# Specify the binary location (replace with your Chrome executable path)
chrome_options.binary_location = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe"

# Launch the driver
#driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome()

In [4]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
DUB
Where to?
MAD
----------
From which city?
MAD
Where to?
DUB
----------
From which city?
DUB
Where to?
TFN
----------
From which city?
TFN
Where to?
DUB
----------
From which city?
DUB
Where to?
IBZ
----------
From which city?
IBZ
Where to?
DUB
----------
From which city?
-1

Routes:
DUB => MAD
MAD => DUB
DUB => TFN
TFN => DUB
DUB => IBZ
IBZ => DUB


In [6]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2023-12-11
End Date, Please use YYYY-MM-DD format only 2023-12-17


In [7]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('div',class_='J0g6-operator-text',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('span',class_='JWEO-stops-text')

    for i in stops:
        stops_list.append(i.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='f8F1-price-text')

    for i in price:
        prices.append(i.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='xdW8 xdW8-mod-full-airport')
    for i in duration:
        duration_list.append(i.text)
    return duration_list

In [8]:
# Set the path to the ChromeDriver executable
chromedriver_path = "C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Launch the driver using the Service class
service = Service(chromedriver_path)
#driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome()

for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination", "Duration", "Total stops", "Price", "Date"]
    df = pd.DataFrame(columns=column_names)

    for j in tqdm(range(num_days + 1)):
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            #service = Service(chromedriver_path)
            #driver = webdriver.Chrome(service=service)
            service = Service()
            driver = webdriver.Chrome()

        url = f"https://www.kayak.ie/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)

        # click show more button to get all flights
        try:
            show_more_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="ULvh-button show-more-button"]'))
            )
            show_more_button.click()
            print("Button pressed")
        except NoSuchElementException:
            print("Show more button not found. Skipping to the next iteration.")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        print(airlines)
        print(total_stops)
        print(prices)
        print(duration)
        new_data = pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops': total_stops,
            'Price': prices,
            'Date': start_date + j
        })
        df = pd.concat([df, new_data], ignore_index=True, axis=0)

    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n', '', regex=True)
    df = df.reset_index(drop=True)

    # save data as a CSV file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv', index=False)
    print(f"Successfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv")

driver.quit()

  0%|          | 0/7 [00:00<?, ?it/s]

Button pressed


 14%|█▍        | 1/7 [00:26<02:36, 26.08s/it]

['Aer Lingus', 'Iberia Express', 'Transavia, Renfe', 'Vueling', 'Iberia', 'Aer Lingus', 'Aer Lingus, Iberia Express', 'Aer Lingus, Air Europa', 'Aer Lingus, Iberia', 'Aer Lingus, easyJet', 'TAP AIR PORTUGAL, easyJet', 'KLM', 'TAP AIR PORTUGAL', 'TAP AIR PORTUGAL', 'KLM', 'Aer Lingus, Air Europa', 'TAP AIR PORTUGAL']
['direct', 'direct', '2 stops', 'direct', 'direct', 'direct', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€251', '€242', '€143', '€244', '€246', '€251', '€183', '€202', '€189', '€175', '€240', '€347', '€290', '€228', '€498', '€200', '€270']
['2h 35m', '2h 35m', '17h 19m', '2h 35m', '2h 35m', '2h 35m', '7h 15m', '5h 35m', '7h 15m', '9h 10m', '5h 15m', '5h 05m', '5h 50m', '8h 10m', '5h 10m', '10h 05m', '8h 35m']
Button pressed


 29%|██▊       | 2/7 [00:42<01:42, 20.48s/it]

...

['Vueling', 'Vueling, Aer Lingus', 'Vueling', 'Vueling, Aer Lingus', 'Vueling, Transavia France', 'Vueling, Aer Lingus', 'CitizenPlane, Aer Lingus', 'Vueling, Aer Lingus', 'Iberia', 'Vueling', 'Iberia Express', 'Iberia, Iberia Express', 'British Airways', 'British Airways', 'Iberia, Aer Lingus']
['1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€53', '€64', '€118', '€111', '€135', '€101', '€152', '€101', '€78', '€169', '€69', '€73', '€235', '€224', '€239']
['8h 05m', '6h 40m', '4h 50m', '6h 10m', '5h 30m', '8h 20m', '6h 10m', '9h 35m', '14h 00m', '6h 35m', '14h 00m', '14h 00m', '6h 20m', '7h 45m', '5h 30m']
Button pressed


 86%|████████▌ | 6/7 [01:51<00:17, 17.63s/it]

['Iberia', 'Vueling', 'Vueling', 'British Airways', 'Vueling, Aer Lingus', 'Vueling, TAP AIR PORTUGAL', 'Iberia', 'Transavia, Aer Lingus', 'Vueling, Aer Lingus', 'Vueling', 'KLM', 'Iberia', 'Air France', 'CitizenPlane, Aer Lingus', 'British Airways', 'Vueling, TAP AIR PORTUGAL']
['1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '2 stops']
['€108', '€46', '€122', '€170', '€142', '€97', '€190', '€227', '€83', '€106', '€259', '€70', '€271', '€125', '€270', '€105']
['5h 55m', '23h 40m', '7h 20m', '5h 55m', '6h 40m', '10h 15m', '10h 10m', '5h 50m', '18h 00m', '16h 20m', '8h 30m', '21h 35m', '8h 35m', '18h 00m', '9h 20m', '18h 30m']
Button pressed


100%|██████████| 7/7 [02:08<00:00, 18.42s/it]

['Vueling', 'Vueling', 'Iberia Express', 'Iberia Express, Iberia', 'Iberia, Iberia Express', 'Iberia Express, Aer Lingus', 'Iberia', 'Vueling, Aer Lingus', 'Iberia, Aer Lingus', 'Vueling', 'Vueling', 'Air Europa, Aer Lingus', 'Transavia, Aer Lingus', 'Iberia', 'Vueling, Aer Lingus']
['1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€43', '€63', '€103', '€106', '€106', '€91', '€109', '€95', '€94', '€63', '€173', '€75', '€214', '€260', '€218']
['6h 15m', '9h 05m', '5h 55m', '5h 55m', '5h 55m', '6h 45m', '5h 55m', '6h 40m', '6h 45m', '12h 20m', '5h 55m', '9h 25m', '5h 40m', '6h 15m', '6h 20m']
Successfully saved IBZ => DUB route as IBZ_DUB.csv
